In [0]:

tables = {"schema":"/mte", "type_raw":"/usr", "type_folder":"/oni", "file_folder":"/rais", "file_subfolder":"/rais_estabelecimentos_2018_2023","prm_path":"/prm/usr/oni/mte/rais_publica_estabelecimento/FIEC_mte_rais_publica_estabelecimento_mapeamento_unificado_trusted.xlsx"}


adf = { "adf_factory_name": "cnibigdatafactory", "adf_pipeline_name": "raw_trs_tb_email", "adf_pipeline_run_id": "61fc4f3c-c592-426d-bb36-c85cb184bb82", "adf_trigger_id": "92abb4ec-2b1f-44e0-8245-7bc165f91016", "adf_trigger_name": "92abb4ec-2b1f-44e0-8245-7bc165f91016", "adf_trigger_time": "2024-05-07T00:58:48.0960873Z", "adf_trigger_type": "PipelineActivity" }

dls = {"folders":{"landing":"/tmp/dev/lnd","error":"/tmp/dev/err","archive":"/tmp/dev/ach","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","trusted":"/tmp/dev/trs","business":"/tmp/dev/biz","prm":"/tmp/dev/prm","historico":"/tmp/dev/hst"},"path_prefix":"tmp","uld":{"folders":{"landing":"/tmp/dev/uld","error":"/tmp/dev/err","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","archive":"/tmp/dev/ach"},"systems":{"raw":"usr"},"path_prefix":"/tmp/dev/"},"systems":{"raw":"usr"}}

dbutils.widgets.text("user_parameters", '{"null": "null"}')

dbutils.widgets.text("env", 'dev')

dbutils.widgets.text("storage", '{"url": "https://cnibigdatadlsgen2.dfs.core.windows.net", "container": "datalake"}')

In [0]:
# Biblioteca cni_connectors, que dá acesso aos dados no datalake
from cni_connectors import adls_connector as adls_conn

# A biblioteca criada para facilitar a declaração dos testes. É necessário importá-la.
### from datatest.gx_context_provider import GXContextProvider

# Essa declaração é necessária em todos os notebooks. Somente através dela podemos acessar os dados no datalake.
var_adls_uri, notebook_params = adls_conn.connect_adls()

from pyspark.sql.functions import udf, from_utc_timestamp, current_timestamp, lit, input_file_name, monotonically_increasing_id, substring_index
from trs_control_field import trs_control_field as tcf
import pyspark.sql.functions as f
import crawler.functions as cf
from pyspark.sql import SparkSession
import time
import pandas as pd
from pyspark.sql.functions import col, when, explode, lit
import json
from unicodedata import normalize 
import datetime
import re
from core.string_utils import normalize_replace
from pyspark.sql.functions import concat, lit, col, substring

In [0]:
# Esses são os dicionários de configuração da transformação enviados pelo ADF e acessados via widgets. Os diretórios de origem e destino das tabelas são compostos por valores em 'dls' e 'tables'.
# Parametros necessario para na ingestão

#tables = notebook_params.var_tables
#dls = notebook_params.var_dls
#adf = notebook_params.var_adf

In [0]:
raw = dls['folders']['raw']
print(raw)

trs = dls['folders']['trusted']
print(trs)

usr = dls['systems']['raw']
print(usr)

/tmp/dev/raw
/tmp/dev/trs
usr


In [0]:
raw_path = "{raw}{type_raw}{type_folder}{schema}{file_folder}{file_subfolder}".format(raw=raw,type_raw=tables['type_raw'], type_folder=tables['type_folder'], schema=tables['schema'], file_folder=tables['file_folder'], file_subfolder=tables['file_subfolder'])
adl_raw = f"{var_adls_uri}{raw_path}"
adl_raw

Out[60]: 'abfss://datalake@[REDACTED].dfs.core.windows.net/tmp/dev/raw/usr/oni/mte/rais/rais_estabelecimentos_2018_2023'

In [0]:
trs_path = "{trs}{type_folder}{schema}{file_folder}{file_subfolder}".format(trs=trs, type_folder=tables['type_folder'], schema=tables['schema'], file_folder=tables['file_folder'], file_subfolder=tables['file_subfolder'])
adl_trs = f"{var_adls_uri}{trs_path}"
adl_trs

Out[61]: 'abfss://datalake@[REDACTED].dfs.core.windows.net/tmp/dev/trs/oni/mte/rais/rais_estabelecimentos_2018_2023'

In [0]:
# prm_path = tables['prm_path']
prm_path = '/tmp/dev/prm/usr/oni/mte/rais_estabelecimento_2018_2023/mte_rais_estabelecimento_mapeamento_unificado_trusted.xlsx'
prm_path

Out[62]: '/tmp/dev/prm/usr/oni/mte/rais_estabelecimento_2018_2023/mte_rais_estabelecimento_mapeamento_unificado_trusted.xlsx'

In [0]:
headers = {
  'name_header': 'Campo Origem',
  'pos_header': 'C',
  'pos_org': 'C',
  'pos_dst': 'E',
  'pos_type': 'F'
}
var_prm_dict = cf.parse_ba_doc(dbutils, prm_path, headers=headers)

In [0]:
df = spark.read.parquet(adl_raw)

In [0]:
def __transform_columns(var_prm_dict__sheet_name: dict):
  for org, dst, _type in var_prm_dict__sheet_name:
    if org == 'N/A':
      yield f.lit(None).cast(_type).alias(dst)
    else:
      yield f.col(org).cast(_type).alias(dst)

In [0]:
# sheet_name = table['table'].upper()
sheet_name = 'RAIS_Identifiad_ESTABELECIMENTO'

cf.check_ba_doc(df, parse_ba=var_prm_dict, sheet=sheet_name)
df = df.select(*__transform_columns(var_prm_dict[sheet_name]))

In [0]:
# var_prm_dict

In [0]:
# Limpar espaços extras nas colunas do tipo string:
for column in df.columns:
  if df.select(column).dtypes[0][1] == 'string':
    df = df.withColumn(column, f.trim(f.col(column)))

# Replace empty strings with null values:
df = df.replace('', None)

In [0]:
# Replace "{ñ class}" and variations with null values:
values_to_replace = [
  '{ñ class}',
  '{ñ class',#
  '{ñ clas',
  '{ñ cla',#
  '{ñ cl',
  '{ñ c',
  '{ñ'
]
for value in values_to_replace:
  df = df.replace(value, None)

In [0]:
# Extract the first two digits
df = df.withColumn("SG_UF", substring(col("CD_MUNICIPIO").cast("string"), 1, 2))

In [0]:
map_dict = {
  '11': 'RO',
  '12': 'AC',
  '13': 'AM',
  '14': 'RR',
  '15': 'PA',
  '16': 'AP',
  '17': 'TO',
  '21': 'MA',
  '22': 'PI',
  '23': 'CE',
  '24': 'RN',
  '25': 'PB',
  '26': 'PE',
  '27': 'AL',
  '28': 'SE',
  '29': 'BA',
  '31': 'MG',
  '32': 'ES',
  '33': 'RJ',
  '35': 'SP',
  '41': 'PR',
  '42': 'SC',
  '43': 'RS',
  '50': 'MS',
  '51': 'MT',
  '52': 'GO',
  '53': 'DF'
}
df = df.replace(map_dict, subset=['SG_UF'])

In [0]:
map_dict = {
  '01': 'ZERO',
  '02': 'ATE 4',
  '03': 'DE 5 A 9',
  '04': 'DE 10 A 19',
  '05': 'DE 20 A 49',
  '06': 'DE 50 A 99',
  '07': 'DE 100 A 249',
  '08': 'DE 250 A 499',
  '09': 'DE 500 A 999',
  '10': '1000 OU MAIS',
  '-1': 'IGNORADO'
}
df = df.replace(map_dict, subset=['DS_TAMANHO_ESTABELECIMENTO'])

In [0]:
df.createOrReplaceTempView("df")

columns_to_select = [col for col in df.columns if col != 'ANO']

columns_to_select.append("ANO AS NR_ANO")

select_query = f"SELECT {', '.join(columns_to_select)} FROM df"

df = spark.sql(select_query)


['CD_CNAE20_CLASSE', 'CD_CNAE20_SUBCLASSE', 'CD_CNAE10_CLASSE', 'CD_TIPO_ESTAB_ID', 'DS_TIPO_ESTAB', 'CD_UF', 'DT_ABERTURA', 'DT_ENCERRAMENTO', 'DT_BAIXA', 'CD_IBGE_SUBSETOR', 'FL_IND_ATIV_ANO', 'FL_IND_CEI_VINCULADO', 'FL_IND_ESTAB_PARTICIPA_PAT', 'FL_IND_RAIS_NEGAT', 'FL_IND_SIMPLES', 'CD_MUNICIPIO', 'CD_NATUREZA_JURIDICA', 'NR_QTD_VINCULOS_ATIVOS', 'NR_QTD_VINCULOS_CLT', 'NR_QTD_VINCULOS_ESTATUTARIOS', 'CD_TAMANHO_ESTABELECIMENTO', 'DS_TAMANHO_ESTABELECIMENTO', 'ID_CNPJ_RAIZ', 'ID_CNPJ_CEI', 'ID_RAZAO_SOCIAL', 'NR_TELEFONE_EMPRE', 'ID_CEPAO_ESTAB', 'NM_BAIRRO', 'NM_EMAIL', 'NM_LOGRADOURO', 'NR_LOGRADOURO', 'SG_UF']
['CD_CNAE20_CLASSE', 'CD_CNAE20_SUBCLASSE', 'CD_CNAE10_CLASSE', 'CD_TIPO_ESTAB_ID', 'DS_TIPO_ESTAB', 'CD_UF', 'DT_ABERTURA', 'DT_ENCERRAMENTO', 'DT_BAIXA', 'CD_IBGE_SUBSETOR', 'FL_IND_ATIV_ANO', 'FL_IND_CEI_VINCULADO', 'FL_IND_ESTAB_PARTICIPA_PAT', 'FL_IND_RAIS_NEGAT', 'FL_IND_SIMPLES', 'CD_MUNICIPIO', 'CD_NATUREZA_JURIDICA', 'NR_QTD_VINCULOS_ATIVOS', 'NR_QTD_VINCULOS_CLT

In [0]:
df = tcf.add_control_fields(df, adf)
df.write.mode('overwrite').partitionBy('NR_ANO').parquet(path=adl_trs)